<a href="https://colab.research.google.com/github/kani215/ML_study/blob/main/ml7_%EB%B2%A0%EC%9D%B4%EC%A7%80%EC%95%88_%EB%B6%84%EB%A5%98%EA%B8%B0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

1. 선언 부분

In [ ]:
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB


"""
나이브 베이즈 테스트 아이리스 데이터 셋 150개 data 3개의 품종 att 존재
데이터 사전 분리
가우시안 나이브베이즈로 활용  - 숫자 형태에 유용
"""

'\n나이브 베이즈 테스트 아이리스 데이터 셋 150개 data 3개의 품종 att 존재\n데이터 사전 분리\n가우시안 나이브베이즈로 활용  - 숫자 형태에 유용\n'

2. 데이터 셋 가져온다.

In [ ]:
X,y = load_iris(return_X_y = True)

3. 데이터 분리

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size = 0.5, random_state=0)

4. 가우시안 나이브베이즈 객체 생성

In [ ]:
nb = GaussianNB()

5. predict

In [ ]:
y_pred = nb.fit(X_train,y_train).predict(X_test)

6. output

In [ ]:
print("Num of mislabeled points from a total %d points : %d"%(X_test.shape[0],(y_test != y_pred).sum() ))

Num of mislabeled points from a total 75 points : 4


실습 - 뉴스 그룹 데이터 분류하기

- 1) 데이터 이해

6개의 att
본문이 있는 data 속성과
데이터의 카테고리 0~19 분류된 target 속성만 사용

In [ ]:
from sklearn.datasets import fetch_20newsgroups
newsdata=fetch_20newsgroups(subset='train')
print(newsdata.keys())

dict_keys(['data', 'filenames', 'target_names', 'target', 'DESCR'])


- 2) key 확인 후 데이터 개수 카테고리 이름 확인

In [ ]:
print (len(newsdata.data), len(newsdata.filenames), len(newsdata.target_names), len(newsdata.target))
print(newsdata.target_names)


11314 11314 20 11314
['alt.atheism', 'comp.graphics', 'comp.os.ms-windows.misc', 'comp.sys.ibm.pc.hardware', 'comp.sys.mac.hardware', 'comp.windows.x', 'misc.forsale', 'rec.autos', 'rec.motorcycles', 'rec.sport.baseball', 'rec.sport.hockey', 'sci.crypt', 'sci.electronics', 'sci.med', 'sci.space', 'soc.religion.christian', 'talk.politics.guns', 'talk.politics.mideast', 'talk.politics.misc', 'talk.religion.misc']


- 3) 데이터 1개 내용 확인
: 스포츠 카에 대한 글 
: 이 글은 7번 target 속성으로 분류 rec.autos 뉴스임

In [ ]:
print(newsdata.data[0])


From: lerxst@wam.umd.edu (where's my thing)
Subject: WHAT car is this!?
Nntp-Posting-Host: rac3.wam.umd.edu
Organization: University of Maryland, College Park
Lines: 15

 I was wondering if anyone out there could enlighten me on this car I saw
the other day. It was a 2-door sports car, looked to be from the late 60s/
early 70s. It was called a Bricklin. The doors were really small. In addition,
the front bumper was separate from the rest of the body. This is 
all I know. If anyone can tellme a model name, engine specs, years
of production, where this car is made, history, or whatever info you
have on this funky looking car, please e-mail.

Thanks,
- IL
   ---- brought to you by your neighborhood Lerxst ----







2. 나이브 베이즈 분류 시작
- 여기서 전처리를 해야하는 데이터는 newsdata.data입니다. 위에서 봤듯이 해당 데이터는 토큰화가 전혀 되어있지 않습니다. 나이브 베이즈 분류를 위해서는 데이터를 BoW로 만들어줄 필요가 있습니다.

- 여기서는 입력한 텍스트를 자동으로 BoW로 만드는 CountVectorizer 사용

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.naive_bayes import MultinomialNB # 다항분포 나이브 베이즈 모델
from sklearn.metrics import accuracy_score #정확도 계산

dtmvector = CountVectorizer()
X_train_dtm = dtmvector.fit_transform(newsdata.data)
print(X_train_dtm.shape)

(11314, 130107)


- 11,314는 훈련용 샘플의 개수이고 DTM 관점에서는 문서의 수가 되겠습니다. 130,107은 전체 훈련 데이터에 등장한 단어의 수


In [ ]:
tfidf_transformer = TfidfTransformer()
tfidfv = tfidf_transformer.fit_transform(X_train_dtm)
print(tfidfv.shape)

(11314, 130107)


- TF-IDF 행렬이 만들어졌습니다. 이제 본격적으로 나이브 베이즈 분류를 수행해보겠습니다. 사이킷런은 나이브 베이즈 모델을 지원하므로, 이를 그대로 갖고와서 사용하겠습니다
- X_train과 y_train에 해당되는 데이터

In [ ]:
mod = MultinomialNB()
mod.fit(tfidfv, newsdata.target)
MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)


MultinomialNB()

- 여기서 alpha=1.0은 라플라스 스무딩이 적용되었음을 의미합니다.



In [ ]:
newsdata_test = fetch_20newsgroups(subset='test', shuffle=True) #테스트 데이터 갖고오기
X_test_dtm = dtmvector.transform(newsdata_test.data) #테스트 데이터를 DTM으로 변환
tfidfv_test = tfidf_transformer.transform(X_test_dtm) #DTM을 TF-IDF 행렬로 변환

predicted = mod.predict(tfidfv_test) #테스트 데이터에 대한 예측
print("정확도:", accuracy_score(newsdata_test.target, predicted)) #예측값과 실제값 비교

정확도: 0.7738980350504514
